In [ ]:
import subprocess

# Replace 'input_video.mp4' with your actual video file name
input_video = 'sanfrancisco_bos_oct31_1.mp4'

# List of time points in seconds
time_points_seconds = [10, 500, 870, 1150, 1430, 1655, 1770]

# Output directory for generated images
output_directory = 'output_images'

# Create the output directory if it doesn't exist
subprocess.run(['mkdir', '-p', output_directory])

# Generate images in JPEG format with incremental suffix
for i, time_point_sec in enumerate(time_points_seconds, start=1):
    output_file = f'{output_directory}/image{i}.jpg'
    subprocess.run([
        'ffmpeg',
        '-ss', f'{time_point_sec:.3f}',  # Specify time point in seconds
        '-i', input_video,
        '-vframes', '1',  # Capture only one frame at the specified time
        '-q:v', '2',  # Quality setting for the output image
        output_file
    ])

print(f'Images generated at time points: {time_points_seconds} seconds')



In [ ]:
import subprocess
import os

# Replace the placeholders with your actual file names and text
image_prefix = 'image'  # Prefix of image file names
image_directory = 'output_images'  # Directory containing image files
audio_file = 'chapter_wise_summary.mp3'
output_file = 'output_video.mp4'

# List of texts corresponding to each image
texts = ['Meeting and Introductions', 
        'Acknowledgments and Pledge of Allegiance', 
        'Meeting Procedures and Communications', 
        'Resolutions and Consent Agenda', 
        'City Governance Charter Reform', 
        'Police Staffing and Redistricting', 
        'Closing Remarks and Adjournment']

# Calculate the duration of each image
total_images = len(texts)
audio_duration = subprocess.run(['ffprobe', '-i', audio_file, '-show_entries', 'format=duration', '-v', 'quiet', '-of', 'csv=p=0'], capture_output=True, text=True)
audio_duration = float(audio_duration.stdout.strip())
image_duration = audio_duration / total_images

# Calculate the correct framerate
framerate = total_images / audio_duration

# Build FFmpeg command
ffmpeg_cmd = [
    'ffmpeg',
    '-framerate', f'{framerate}',  # Set the correct framerate
    '-start_number', '1',
    '-i', os.path.join(image_directory, 'image%d.jpg'),
    '-i', audio_file,
]

# Add drawtext filter for each text
for i, text in enumerate(texts):
    start_time = i * image_duration
    ffmpeg_cmd.extend([
        '-vf', f"drawtext=text='{text}':fontsize=24:x=(w-text_w)/2:y=(h-text_h)/2:enable='between(t,{start_time},{start_time+image_duration})'",
    ])

# Add audio and output options
ffmpeg_cmd.extend([
    '-c:a', 'aac',
    '-strict', 'experimental',
    '-b:a', '192k',
    '-t', str(audio_duration),  # Set the duration of the output video to match the audio duration
    output_file,
])

# Run FFmpeg command
subprocess.run(ffmpeg_cmd)